# MS Calculations

Notebook to crunch numbers for the MS.

by Cascade Tuholske 2020.02.23 

Updated 2020.08.27 - CPT
Was run on ERA5 RH with CHIRTS-Daily Tmax from ERA5

In [2]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import glob
import matplotlib.dates as mdates

In [ ]:
#### Regressions, no intercept addition is needed because we're using SK LEARN HERE 

def lm_func(df, col):
    
    "simple linear model of a time series data, returns coef"
    
    # Get Data
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[col].sum()).reshape((-1, 1))

    # Add Intercept
    X_year_2 = sm.add_constant(X_year)

    # Regress
    model = sm.OLS(Y_stats, X_year_2).fit() 
        
    coef = int(model.params[1])
    #coef = int(coef)
            
    # R2 and P
    r2 = model.rsquared_adj
    p = model.pvalues[0]
    
    return coef, round(r2, 2), round(p, 3)

In [ ]:
#### Load Data
DATA = 'WBGT32_1D' # UPDATE 

# file paths
DATA_IN = "/home/cascade/projects/UrbanHeat/data/"  
FIG_OUT = "/home/cascade/projects/UrbanHeat/figures/"
FN_IN = 'processed/PNAS-DATA-v2/'+DATA+'_EXP.json'
HI_STATS = pd.read_json(DATA_IN+FN_IN, orient = 'split')

# Set scale
scale = 10**9


In [ ]:
# Drop cites where 1983 had 1 day and none elsewhere

print(len(HI_STATS))
only83 = HI_STATS.groupby('ID_HDC_G0')['tot_days'].sum() == 1 # sum up total days and find those with 1 day
only83 = list(only83[only83 == True].index) # make a list of IDs
sub = HI_STATS[HI_STATS['ID_HDC_G0'].isin(only83)] # subset those IDs
bad_ids = sub[(sub['year'] == 1983) & (sub['tot_days'] == 1)] # drop those from 1983 only
drop_list = list(bad_ids['ID_HDC_G0']) # make a list
HI_STATS= HI_STATS[~HI_STATS['ID_HDC_G0'].isin(drop_list)] # drop those from the list
print(len(HI_STATS))

In [ ]:
#### Add In Meta Data (e.g. geographic data)
meta_fn = DATA_IN+'interim/GHS-UCDB-IDS.csv'
meta_data = pd.read_csv(meta_fn)

#### Merge in meta
HI_STATS = HI_STATS.merge(meta_data, on = 'ID_HDC_G0', how = 'left')

# Global Trends

In [ ]:
#### Total Change in people Days
data = HI_STATS.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)


In [ ]:
#### Rate of change
data = HI_STATS
coef, r2, p = lm_func(data, 'people_days')
print('annual increase in people days ', 'was', coef/10**9, ' p=', p)
coef1, r21, p1 = lm_func(data, 'people_days_heat')
print('annual increase in people days heat ', 'was', coef1/10**9, ' p=', p)
coef2, r22, p2 = lm_func(data, 'people_days_pop')
print('annual increase in people days pop ', 'was', coef2/10**9, ' p=', p)
print('attrib heat ', 'was', coef1 / coef *100, ' p=', p, '\n')

In [ ]:
#### Pct Pday Annual Increase from Heat
coef_pdays, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days') # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat') # regreas heat

print('warming is what pct of total?', coef_heat/coef_pdays *100)

# City-level

#### Largest cities compared to global total

In [ ]:
#### Top cities
cities = pd.read_csv(DATA_PATH+'processed/PNAS-DATA-v2/WBGT32_1D_EXP-TOP50.csv')

In [ ]:
top = cities.sort_values('coef_pdays', ascending = False).head(25) # get the top ten cities


In [ ]:
# What pct of the global annual increase comes from the top ten cities?
ans = top['coef_pdays'].sum() / coef
print('Top 25 cities of total annual increase', ans * 100)

#### Pdays

In [ ]:
city_coefs = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_TREND_EXP05.json', orient = 'split')
GHS = gpd.read_file('/home/cascade/projects/UrbanHeat/data/raw/GHS_UCDB/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp')

In [ ]:
len(city_coefs)

In [ ]:
#### Number of cities w/ sig increase in exposure?
print('The pct of cities w/ increases in exposure: ', len(city_coefs)/len(GHS)*100)


In [ ]:
len(city_coefs)

In [ ]:
ans = len(city_coefs[(city_coefs['GCPNT_LAT'] < 23.5) & (city_coefs['GCPNT_LAT'] > -23.5)]) / len(city_coefs)
print('what pct of pday cities are low lat?', ans*100)

In [ ]:
print('what pct of global pop are cities with sig pdays?')

In [ ]:
def country_search(country, data_set):
    "what pct of cities had a p-day increase?"
    print('Num of Cities in '+country+' ', len(data_set[data_set['CTR_MN_NM'] == country]) / len(GHS[GHS['CTR_MN_NM'] == country]) *100)

In [ ]:
data_set = city_coefs

In [ ]:
country_search('Senegal', data_set)

In [ ]:
country_search('Nigeria', data_set)

In [ ]:
country_search('India', data_set)

#### Pct of global population exposured

In [ ]:
city_coefs = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_TREND_EXP05.json', orient = 'split')

In [ ]:
pop = pd.read_csv(DATA_IN+'interim/GHS-UCDB-Interp.csv')
p16 = pop[['ID_HDC_G0', 'P2016']]

In [ ]:
len(p16)

In [ ]:
pdays_pop = pd.merge(city_coefs[['ID_HDC_G0']], p16, on = 'ID_HDC_G0', how = 'inner')

In [ ]:
ans = pdays_pop['P2016'].sum() / p16['P2016'].sum() * 100 
print('What is the global urban population in 2016', p16['P2016'].sum())
print('How many people live in cities with increasing exp in 2016', pdays_pop['P2016'].sum())
print('What pct of total urban pop has sig increase exp in 2015', ans)

In [ ]:
# From UN-DESA 2018 estimates for total global pop in 2015
ans =  pdays_pop['P2016'].sum() / 7383009000 * 100
print('What pct of total world pop has sig increase exp in 2016', ans)

In [ ]:
# UN-DESA Urban pop in 2015 was  3 981 498
p16['P2016'].sum()

#### Total Heat Days 

In [ ]:
city_totdays = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_TREND_HEATP05.json', orient = 'split')

In [ ]:
print('What pct of all cities had sig increase in days/yr > WBGT32 C ?')
print(len(city_totdays)/len(GHS))
print(len(city_totdays))

In [ ]:
print('What pct of all cities >1 day / yr in days/yr > WBGT32 C ?')
print(len(city_totdays[city_totdays['coef_totDays'] >= 1])/len(GHS))
print(len(city_totdays[city_totdays['coef_totDays'] >= 1]))

In [ ]:
## How many cities day increase per year ... 1, 3
top = len(city_totdays)
bottom = len(city_totdays[city_totdays['coef_totDays'] >= 2])

In [ ]:
print(top)
print(bottom)

In [ ]:
## What are some big cities?
hot50 = city_totdays[(city_totdays['coef_totDays'] >= 1.5) * city_totdays['P2016'] >= 5*10**5][['coef_totDays', 'UC_NM_MN']].sort_values('coef_totDays')

In [ ]:
len(hot50)

In [ ]:
#### Columbo & San Salvador & Conakry
print('Conakry tot days:', city_totdays[city_totdays['ID_HDC_G0'] == 1502]['coef_totDays'].values)
print('columbo tot days:', city_totdays[city_totdays['ID_HDC_G0'] == 8835]['coef_totDays'].values)
print('San Sal tot days:', city_totdays[city_totdays['ID_HDC_G0'] == 321]['coef_totDays'].values)

#### Dehli & Kolkata 

In [ ]:
# Delhi 6955 & Kolkata 9691
K = city_coefs[city_coefs['ID_HDC_G0']== 9691]
D = city_coefs[city_coefs['ID_HDC_G0']== 6955]

In [ ]:
print('Share of heat Kolkata', K.coef_heat / K.coef_pdays * 100)

In [ ]:
print('Share of heat Delhi', D.coef_heat / D.coef_pdays * 100)

#### Populations of specific cities

In [ ]:
pop = pd.read_csv(DATA_IN+'interim/GHS-UCDB-Interp.csv')

In [ ]:
# 9691, Kolkata 1998
# 2046, Paris 2003
# 4417, Aleppo 2010

In [ ]:
ans = pop[pop['ID_HDC_G0'] == 9691]['P2015'] / 10**3
print('Pop of Kolkata in 1998', ans)

# Regional Trends

In [ ]:
#### Annual Rates

scale = 10**6
geog = 'sub-region'

for label in np.unique(HI_STATS[geog]):
    label = label
    data = HI_STATS[HI_STATS[geog] == label]
    
    #### Rate of change
    coef, r2, p = lm_func(data, 'people_days')
    print('annual increase in people days '+label, 'was', coef/scale, ' p=', p)
    coef1, r21, p1 = lm_func(data, 'people_days_heat')
    print('annual increase in people days heat '+label, 'was', coef1/scale, ' p=', p)
    coef2, r22, p2 = lm_func(data, 'people_days_pop')
    print('annual increase in people days pop '+label, 'was', coef2/scale, ' p=', p)
    print('attrib heat '+label, 'was', coef1 / coef *100, ' p=', p, '\n')
  

In [ ]:
#### Trends for Africa, N & SS
geog = 'region'
location = 'Africa'
data = HI_STATS[HI_STATS[geog] == location]
print(location)

#### Total Change in people Days
data = data.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)



In [ ]:
#### S Asia as pct of total  global = 5.245146271 B 

print('pct of total pdays from S Asia is ', 1899.70765 / 10**3 / 5.245146271 * 100)

In [ ]:
#### Median Slope
region = 'Europe'
col = 'coef_heat'
geog = 'region'
scale = 10**3
result = city_coefs[city_coefs[geog]== region][col].median()
print(region, col, 'is ', result/scale)

# Heat Waves

- 9691 Kolkata 1998
- 2046 Paris 2003
- 4417, Aleppo 2010

In [3]:
def make_data(dir_in, geog, location):
    """Function makes data to plot daily city-level HI Max and average
    Args:
        dir_in = directory to get data
        geog = column for geography, city-level = 'ID_HDC_G0'
        location = usually a city id
    """
    
    fn_list = sorted(glob.glob(dir_in+'*.csv')) # get data
    df_out = pd.DataFrame() # to write dataframe
    
     # get leap year cols from 2016
    hi16 = pd.read_csv(fn_list[33]) 
    cols = list(hi16.iloc[:,3:].columns)
    cols = [year[5:] for year in cols] # cols for data frame
    
    temp_list = [] # empty list for temps
    
    # loop through dir and get data
    for i, fn in enumerate(fn_list):
        df = pd.read_csv(fn) # open data frame
        year_label = [(df.columns[3]).split('.')[0]] # get year
        row = df[df[geog] == location]
        temp = row.iloc[:,3:] # get only temp columns
        
        # add in col for leap years
        if temp.shape[1] == 365:
            temp.insert(loc = 59, column = year_label[0]+'.02.29', value = np.nan, allow_duplicates=False)

        # Set Index & Columns
        temp.index = year_label
        temp.columns = cols # revalue to m.d
    
        # add to list
        temp_list.append(temp)
    
    df_out = pd.concat(temp_list) # make one big dataframe
    
    return df_out

In [4]:
def plot_data(df, year, start, end):#, start, end):
    """ Make the data for a plot
    Args: 
        df = df w/ daily HI max for a given city
        year = year you want to plot against average
        start = start of plot in julian days (e.g 1 - 365/366)
        end = end of plot in julian days
    """

    # Deal with leap year
    if year % 4 !=0:
        df.drop(columns ='02.29', inplace = True)
    
    # Subset data
    start = start - 1 # zero indexing 
    subset = df.iloc[:,start:end]
    
    # HI Max for year
    hi_year = subset.loc[str(year)]
    
    # make 34-avg daily hi
    means = subset.mean(axis = 0)
    
    # make colums to date time
    cols = pd.to_datetime([str(year)+'.'+date for date in hi_year.index])
    
    return hi_year, means, cols

In [71]:
# Find Heat Wave From All DATA
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out

# open heat events data
events_fn = '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI461_1D_STATS.json'
events = pd.read_json(events_fn, orient = 'split')

# [4417, 'Aleppo'] [2046, 'Paris'] [9691, 'Kolkata'] 
city = select_city_year(events, 4417, 2010)
city

,ID_HDC_G0,year,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,UID
148,4417,2010,666,2,46.356890,0.256890,0.513780,"[2010.07.19, 2010.07.20]","[0.3747319333, 0.13904817090000002]","[46.4747319333, 46.2390481709]",UID-1718681
149,4417,2010,666,1,47.189391,1.089391,1.089391,[2010.07.22],[1.089390514],[47.189390514],UID-1718682
150,4417,2010,666,1,46.610048,0.510048,0.510048,[2010.07.26],[0.5100479559000001],[46.6100479559],UID-1718683
151,4417,2010,666,9,51.839383,5.739383,51.654447,"[2010.07.29, 2010.07.30, 2010.07.31, 2010.08.0...","[3.2814992021, 7.1096257613, 12.5753128385, 6....","[49.3814992021, 53.2096257613, 58.6753128385, ...",UID-1718684
152,4417,2010,666,7,53.780789,7.680789,53.765526,"[2010.08.10, 2010.08.11, 2010.08.12, 2010.08.1...","[4.7263495980000005, 9.0482525407, 8.917087211...","[50.826349598, 55.1482525407, 55.0170872119, 5...",UID-1718685
153,4417,2010,666,3,49.940370,3.840370,11.521111,"[2010.08.19, 2010.08.20, 2010.08.21]","[5.0217016262, 3.2292439878, 3.2701651029]","[51.1217016262, 49.3292439878, 49.3701651029]",UID-1718686
154,4417,2010,666,3,47.528756,1.428756,4.286268,"[2010.08.23, 2010.08.24, 2010.08.25]","[1.0340144974, 2.3068767788, 0.9453762832]","[47.1340144974, 48.4068767788, 47.0453762832]",UID-1718687
155,4417,2010,666,1,47.976825,1.876825,1.876825,[2010.09.05],[1.8768247615],[47.9768247615],UID-1718688
156,4417,2010,666,3,46.524747,0.424747,1.274240,"[2010.09.09, 2010.09.10, 2010.09.11]","[0.4653870438, 0.6443832787, 0.16447000450000002]","[46.5653870438, 46.7443832787, 46.2644700045]",UID-1718689


In [46]:
# Select event and compare to long term means
# Kolkata 1998 UID-2920905, Aleppo 2010 
event_id = 'UID-2920905'
event = city[city['UID'] == event_id]
tmax = list(event['tmax'])[0]
dates = list(event['event_dates'])[0]

In [21]:
#### Heat Index Data
data = 'HI'
DATA_IN = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/'+data+'/' # output from avg temp
t = 46.1

In [68]:
# Get long term means
# Args
#[4417, 'Aleppo'] 2010 [2046, 'Paris'] 2003 [9691, 'Kolkata'] 1998 ['6955, Dehli']

# Args
city_list = [4417, 'Aleppo'] 
year = 2010

# April 1 to Sep 30 (Use Julian Days), or 1 - 182 lagos
start = 91 
end = 273

# Make Data 
df = make_data(DATA_IN, 'ID_HDC_G0', city_list[0])
years, means, cols = plot_data(df, year, start, end)

In [70]:
cols

DatetimeIndex(['2010-04-01', '2010-04-02', '2010-04-03', '2010-04-04',
               '2010-04-05', '2010-04-06', '2010-04-07', '2010-04-08',
               '2010-04-09', '2010-04-10',
               ...
               '2010-09-21', '2010-09-22', '2010-09-23', '2010-09-24',
               '2010-09-25', '2010-09-26', '2010-09-27', '2010-09-28',
               '2010-09-29', '2010-09-30'],
              dtype='datetime64[ns]', length=183, freq=None)

In [35]:
# make a df
df = pd.DataFrame()
df['dates'] = cols
df['max']  = years.to_list()
df['avg'] = means.to_list()
df['dif'] = df['max'] - df['avg']

In [37]:
type(df['dates'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [40]:
type(dates[0][0])

str

In [48]:
# start and end of heat wave
start = dates[0]
end = dates[-1]

In [62]:
out = df[(df['dates'] >= start) & (df['dates'] <= end)]

In [66]:
out.max()

dates    1998-06-14 00:00:00
max                  66.0351
avg                  55.5479
dif                  12.5468
dtype: object

In [67]:
out[out['dif'] == out['dif'].max()]

,dates,max,avg,dif
68,1998-06-08,65.644533,53.097732,12.546801


# Check out cities

In [ ]:
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out


In [ ]:
city = select_city_year(ALL_DATA, 4417, 2010)
city

# LA

In [ ]:
from glob import glob 
import os

In [ ]:
dir_list = '/home/cascade/projects/UrbanHeat/data/interim/ERA5_HI/'


In [ ]:
fn = 'GHS-ERA5-HI_2009.csv'
data = pd.read_csv(dir_list+fn)
city = data[data['ID_HDC_G0'] == 14]

In [ ]:
temps_list = []
dates_list = []
for fn in sorted(os.listdir(dir_list)):
    data = pd.read_csv(dir_list+fn)
    city = data[data['ID_HDC_G0'] == 14]
    dates = list(city.iloc[:,3:])
    temps = list(city.iloc[:,3:].values[0])
    dates_list.extend(dates)
    temps_list.extend(temps)

In [ ]:
len(dates_list)

# Africa

In [ ]:
#### Dependencies
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.dates as mdates

In [ ]:
def make_data(dir_in, geog, location):
    """Function makes data to plot daily city-level HI Max and average
    Args:
        dir_in = directory to get data
        geog = column for geography, city-level = 'ID_HDC_G0'
        location = usually a city id
    """
    
    fn_list = sorted(glob.glob(dir_in+'*.csv')) # get data
    df_out = pd.DataFrame() # to write dataframe
    
     # get leap year cols from 2016
    hi16 = pd.read_csv(fn_list[33]) 
    cols = list(hi16.iloc[:,3:].columns)
    cols = [year[5:] for year in cols] # cols for data frame
    
    temp_list = [] # empty list for temps
    
    # loop through dir and get data
    for i, fn in enumerate(fn_list):
        df = pd.read_csv(fn) # open data frame
        year_label = [(df.columns[3]).split('.')[0]] # get year
        row = df[df[geog] == location]
        temp = row.iloc[:,3:] # get only temp columns
        
        # add in col for leap years
        if temp.shape[1] == 365:
            temp.insert(loc = 59, column = year_label[0]+'.02.29', value = np.nan, allow_duplicates=False)

        # Set Index & Columns
        temp.index = year_label
        temp.columns = cols # revalue to m.d
    
        # add to list
        temp_list.append(temp)
    
    df_out = pd.concat(temp_list) # make one big dataframe
    
    return df_out

In [ ]:
def plot_data(df, year, start, end):#, start, end):
    """ Make the data for a plot
    Args: 
        df = df w/ daily HI max for a given city
        year = year you want to plot against average
        start = start of plot in julian days (e.g 1 - 365/366)
        end = end of plot in julian days
    """

    # Deal with leap year
    if year % 4 !=0:
        df.drop(columns ='02.29', inplace = True)
    
    # Subset data
    start = start - 1 # zero indexing 
    subset = df.iloc[:,start:end]
    
    # HI Max for year
    hi_year = subset.loc[str(year)]
    
    # make 34-avg daily hi
    means = subset.mean(axis = 0)
    
    # make colums to date time
    cols = pd.to_datetime([str(year)+'.'+date for date in hi_year.index])
    
    return hi_year, means, cols

In [ ]:
# Args
#[3342, 'Cairo'] #[1910, 'Accra'] 3268, Cape Town
#[4417, 'Aleppo'] 2010 [2046, 'Paris'] 2003 [9691, 'Kolkata'] 1998 ['6955, Dehli']

# Args
city_list = [9691, 'Kolkata'] 
year = 1998
font_size = 10


# April 1 to Sep 30 (Use Julian Days), or 1 - 182 lagos
start = 91 
end = 273

# Labels
hi_label = str(year)+' '+data
labels =  ['avg. '+data, hi_label, str(t)+''+DS+'C']  # <<<<<------------ Be sure to update! 

# Make Data 
df = make_data(DATA_IN, 'ID_HDC_G0', city_list[0])
years, means, cols = plot_data(df, year, start, end)

# Trend First vs. Second Half of Data

In [ ]:
data.head()

In [ ]:
#### Share of exposure due to heat by 17 year split

## 1983 - 1999
data1 = HI_STATS[(HI_STATS['year'] >= 1983) & (HI_STATS['year'] < 2000)]
coef1pop , r21pop, p1pop  = lm_func(data1 , 'people_days_pop')
coef1heat , r21heat, p1heat = lm_func(data1 , 'people_days_heat')

years = list(np.unique(data1['year']))
plt.plot(years, data1.groupby('year')['people_days_heat'].sum())
sns.regplot(years, data1.groupby('year')['people_days_heat'].sum(), 
            color = 'blue', scatter = False, truncate = True)

## 2000 - 2016
data2 = HI_STATS[(HI_STATS['year'] >= 2000) & (HI_STATS['year'] <= 2016)]
coef2heat , r22heat, p2heat = lm_func(data2 , 'people_days_heat')
coef2pop , r22pop, p1pop  = lm_func(data2 , 'people_days_pop')

years = list(np.unique(data1['year']))
plt.plot(years, data2.groupby('year')['people_days_heat'].sum())
sns.regplot(years, data2.groupby('year')['people_days_heat'].sum(), 
            color = 'orange', scatter = False, truncate = True)

In [ ]:
## 2000 - 2016
data2pop = HI_STATS[(HI_STATS['year'] >= 1983) & (HI_STATS['year'] < 2000)]
coef2pop , r22pop, p1pop  = lm_func(data2pop , 'people_days_pop')

data2heat = HI_STATS[(HI_STATS['year'] >= 2000) & (HI_STATS['year'] <= 2016)]
coef2heat , r22heat, p2heat = lm_func(data2heat , 'people_days_heat')


In [ ]:
## Estimates
print('From 83 - 99, contribution from heat was', coef1heat/(coef1pop+coef1heat))
print('From 00 - 16, contribution from heat was', coef2heat/(coef2pop+coef2heat))
print('From 83 - 00, heat was', coef1heat/10**9, round(p1heat, 3))
print('From 00 - 16, heat was', coef2heat/10**9)
print('From 83 - 00, pop was', coef1pop/10**9)
print('From 00 - 16, pop was', coef2pop/10**9)